In [ ]:
# INPUT:
# TODO: Build simple ui or just use terminal for inputs
# TODO: Automatically score scripts that have been found in directory

# SCORING:
# TODO: Find tests
# TODO: Parse files to get necessary info
# TODO: Develop relevant utility functions
# TODO: Make sure to score all trials separately (practices can be scored all at once)
# TODO: All proportion correct should be displayed in percentage format
# TODO: Make sure to average response time
# TODO: Display response times per each response later on
# TODO: Response time correct vs incorrect avgs
# TODO: Score distance using city block method (will be in the nitty gritty area)
# TODO: Score avg distance across trials

# OUTPUT:
# TODO: Open csv for each test scored
# TODO: Show preliminary information first
# TODO: Display more in-depth details further along (horizontal)
# TODO: Display avg distance from correct block (incorrect answers) in the 
# TODO: Place all scored files in scored directory
# TODO: Print LOGS stating what is being done

# FINAL:
# TODO: Migrate .ipynb to .py format and finish syncing script
# TODO: Make script executable?!?!
# TODO: Allow for more in-depth customization in args (argparse)

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import os
import logging as log

class Util:
    def __init__(self) -> None:
        pass

    def findFile(testName=str):

        # Get the absolute path of the current script file
        current_script_path = os.path.abspath(__file__)

        # Get the directory containing the script file
        current_dir = os.path.dirname(current_script_path)

        # Change the current working directory to the script directory
        os.chdir(current_dir)

        # Search for files containing the test name
        for file_name in os.listdir(current_dir):
            if file_name.endswith('.txt'):  # change the extension as needed
                if testName in file_name:
                    file_path = os.path.join(current_dir, file_name)
                    pd.read_csv(file_path)
                    print(file_path)
                    

class InputHandler(Util):
    def __init__(self) -> None:
        super().__init__()


class OutputHandler(Util):
    def __init__(self) -> None:
        super().__init__()

    

myUtil = Util()
myOutputHandler = OutputHandler()


In [13]:
def luScore():
    file = pd.read_csv('1001_FiguralSpeed.txt')
    print(file)
luScore()

    1001\t2\t6\t1\tPRACT\t1\t8000\t500\t2\tFa4_1211\tFa4_2211\t/\t/\t1\t1741\t2023\t4\t5\t15\t38\t21
0    1001\t2\t6\t1\tPRACT\t2\t8000\t500\t2\tFa1_331...                                              
1    1001\t2\t6\t1\tPRACT\t3\t8000\t500\t1\tFa2_232...                                              
2    1001\t2\t6\t1\tPRACT\t4\t8000\t500\t1\tFa3_311...                                              
3    1001\t2\t6\t1\tPRACT\t5\t8000\t500\t1\tFa1_212...                                              
4    1001\t2\t6\t1\tPRACT\t6\t8000\t500\t2\tFa1_221...                                              
..                                                 ...                                              
114  1001\t2\t6\t3\tTRIAL2\t36\t8000\t500\t2\tFc4_1...                                              
115  1001\t2\t6\t3\tTRIAL2\t37\t8000\t500\t1\tFc2_2...                                              
116  1001\t2\t6\t3\tTRIAL2\t38\t8000\t500\t1\tFc4_2...                                     

In [ ]:
def nbScore():
    pd.read_csv()
    pass

nbScore()

In [ ]:
def suScore():
    pass

suScore()

In [ ]:
def stScore():
    pass

stScore()

In [ ]:
def wrScore():
    pass

wrScore()

In [ ]:
def nsScore():
    pass

nsScore()

In [ ]:
def olmScore():
    pass

olmScore()

In [ ]:
def vsScore():
    pass

vsScore()

In [ ]:
def nmScore():
    pass

nmScore()

In [ ]:
def fsScore():
    pass

fsScore()

In [2]:
def msScore():
    pass

msScore